In [14]:
import mne 
import numpy as np

In [ ]:
# Function to generate final text reporting color distribuction
def generate_text(band_distribution):
    text = []
    for color in color2band:
        text.append(f"{color}: {band_distribution[color2band[color]]:.2f}%")
    
    return ', '.join(text)

In [8]:
# Note: for the purpose of demonstration, I used mne package to import the example data. If you are going to process EEG from 
# lsl, you will need to write corresponding code to receive EEG from lsl. The current code should be suitable for offline 
# analysis, though. 

raw_epoch = mne.read_epochs_eeglab('./Example_Data/EEG_Subj_1_Preprocessed.set')
eeg_data = raw_epoch.get_data()

Extracting parameters from /mnt/c/Users/AdminW/Desktop/Research/Emotion_Color_Detection/Example_Data/EEG_Subj_1_Preprocessed.set...
Not setting metadata
45 matching events found
No baseline correction applied
0 projection items activated
Ready.


/tmp/ipykernel_4760/2299928715.py:1: RuntimeWarning: At least one epoch has multiple events. Only the latency of the first event will be retained.
  raw_epoch = mne.read_epochs_eeglab('./Example_Data/EEG_Subj_1_Preprocessed.set')
/tmp/ipykernel_4760/2299928715.py:1: RuntimeWarning: Estimated head radius (0.0 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw_epoch = mne.read_epochs_eeglab('./Example_Data/EEG_Subj_1_Preprocessed.set')


In [11]:
# Data size for the example code: 45 epochs x 224 channels x 3 second (100Hz)
eeg_data.shape

(45, 224, 300)

In [40]:
#sampling rate
fs = 100

# Define EEG bands
eeg_bands = {'Beta_Fast': (20, 30),
             'Beta_Middle': (16, 20),
             'Beta_Low': (12, 16),
             'Alpha_Fast': (12, 14),
             'Alpha_Middle': (9, 12),
             'Alpha_Low': (8, 9),
             'Theta': (4,7),
             'Gamma': (31, 50)}

# Define Color Translation 
color2band = {
    'Red': 'Beta_Fast',
    'Orange': 'Beta_Middle',
    'Yellow': 'Beta_Low',
    'Green': 'Alpha_Fast',
    'Sky Blue': 'Alpha_Middle',
    'Indigo': 'Alpha_Low',
    'Violet': 'Theta',
    'Gold': 'Gamma'
}

In [36]:
num_trial, num_channels = eeg_data.shape[:2]
eeg_band_collection = []

# looping through all the epochs to compute band power distribution. Note that if we are doing real time analysis, 
# we may not have multiple epochs to analyze so please modify accordingly (i.e. by removing the for loop for epochs)
for t in range(num_trial):
    
    # Create dictionary to strore band amplitudes for the epoch
    eeg_band_fft = {
                 'Beta_Fast': [],
                 'Beta_Middle': [],
                 'Beta_Low': [],
                 'Alpha_Fast': [],
                 'Alpha_Middle': [],
                 'Alpha_Low': [],
                 'Theta': [],
                 'Gamma': []}
        
    for c in range(num_channels):
        
        epoch = eeg_data[t][c]
        
        # Get real amplitudes of FFT (only in postive frequencies)
        fft_vals = np.absolute(np.fft.rfft(epoch))

        # Get frequencies for amplitudes in Hz
        fft_freq = np.fft.rfftfreq(len(epoch), 1.0/fs)

        # Take the mean of the fft amplitude for each EEG band
        for band in eeg_bands:  
            freq_ix = np.where((fft_freq >= eeg_bands[band][0]) & 
                               (fft_freq <= eeg_bands[band][1]))[0]
            eeg_band_fft[band].append(np.mean(fft_vals[freq_ix]))
           
    sum_power = 0
    #Compute the average power across all channels 
    for band in eeg_bands:
        eeg_band_fft[band] = np.mean(eeg_band_fft[band])
        sum_power += np.mean(eeg_band_fft[band])
        
    #Compute the average power across all channels 
    for band in eeg_bands:
        eeg_band_fft[band] = (eeg_band_fft[band]/sum_power)*100
        
    eeg_band_collection.append(eeg_band_fft)

In [46]:
# Use generate_text to generate the final text summary, here we use the first epoch as an example
generate_text(eeg_band_collection[0])

'Red: 5.39%, Orange: 8.62%, Yellow: 13.63%, Green: 15.67%, Sky Blue: 12.92%, Indigo: 20.27%, Violet: 21.12%, Gold: 2.37%'